# MP2- Traffic Sign Classifier
Before you start, please read through this tutorial of Pytorch: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py.
The basic structure of this MP will be very similar to the example code in the tutorial. But your neural network model need to be more complicated. 

If any packege is not installed, you shall be able to install them through pip3 install. 

In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io, transform
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, sampler
import torchvision
from torchvision import transforms, utils
import torchvision.utils as vutils
import torchvision.datasets as dset
from torch.autograd import Variable

Set up the parameters. If you have GPU, change the num_gpu. 

In [2]:
# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. The image size of original images vary from 15 to 250. 
#But to train our CNN we must have the fixed size for the inputs.
#All images will be resized to this size using a transformer.
image_size = 32

# Number of training epochs
num_epochs = 15

#number of training classes
num_classes = 43

# Learning rate for optimizers
learning_rate = 0.002

# Number of GPUs available. Use 0 for CPU mode.
num_gpu = 1

# Set random seed for reproducibility
manualSeed = 999
torch.manual_seed(manualSeed)

### Data Preprocessing 
To improve the performance of our CNN, we need to use some data augmentation techniques. We can change the orientation, location, scale, saturation or brightness of the original image to modify the original images or create new training images. 

The easiest way to go is openCV. If you are having problem with ppm format, you can convert the images to other image format (PNG, JPG...) using pillow packege: https://pillow.readthedocs.io/en/latest/handbook/tutorial.html

You are reqiured to test at least 3 techniques and compare the effectiveness(the accuracy improvement). For each technique, you need to generate new images for at least one class of images. For instance, if you originally have 300 training images in class 01, then you need to generate 300 new images and save them into the same folder as old images. Note that it may be helpful to name the new images properly, so if you find your technique is not helpful, you can remove them easier. <br>
You can skip this part and try to have a working neural network first, then come back here to finish it. 

### Load the data
Load the dataset into PyTorch. Assign the path to the training and testing dataset you downloaded previously to the variables "trainingClassifierRoot" and "testClassifierRoot" below. In the beginning you can just use the original data downloaded from website. Later on you will need to process the image before load them.  

In [3]:
##TODO
trainingClassifierRoot = '/home/peixin/mp2_data/Final_Training/Images'
testClassifierRoot = '/home/peixin/mp2_data/Online-Test-sort/'
####
normImgTensor = transforms.Normalize(mean=np.zeros(3), std=np.ones(3))

# Create the dataset
trainClassifierDataset = dset.ImageFolder(root=trainingClassifierRoot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
							   normImgTensor
                           ]))

testClassifierDataset = dset.ImageFolder(root=testClassifierRoot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
							   normImgTensor
                           ]))

# Create the dataloader
trainClassifierLoader = torch.utils.data.DataLoader(trainClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)
testClassifierLoader = torch.utils.data.DataLoader(testClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)

# Decide which device(GPU or CPU) we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and num_gpu > 0) else "cpu")

print ('...')

...


### Design the model
In this part you will design the structure of CNN. The basic syntax is the same as the toy example in PyTorch tutorial. But you definitely need some more complex models to meet the reqiurements. You need to try at least 3 different structures and report their accuracy and running time (how long it would take to go through the test set). The best one is required to achieve 97% accuracy. Describe the CNN structures and compare the results in your report. Feel free to use any existing structure you find on Internet, but citation in your report is needed.<br>
Here is a NN structure that has 98% accuracy. You can start from there: https://chatbotslife.com/german-sign-classification-using-deep-learning-neural-networks-98-8-solution-d05656bf51ad<br>
If you really want to get higher accuracy, here are some state-of-the-art papers for your reference: 
<li>https://arxiv.org/abs/1511.02992</li>
<li>https://www.sciencedirect.com/science/article/pii/S0893608018300054?via%3Dihub</li>
<li>http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf</li>

In [44]:
class TrafficSignClassifier(nn.Module):
    def __init__(self, num_gpu):
        super(TrafficSignClassifier, self).__init__()
        self.ngpu = num_gpu
       
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=(2,2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=(2,2))
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=(2,2))

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=(2,2))
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding=(2,2))
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, padding=(2,2))

        self.pool = nn.MaxPool2d(2, 2)

        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(256)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.relu = nn.ReLU(inplace = True)
        self.linear = nn.Linear(4*4*64, num_classes)
        ####
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x  = self.pool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.linear(x) 
        return x
	
class TrafficSignClassifier1(nn.Module):
	def __init__(self, num_gpu):
		super(TrafficSignClassifier1, self).__init__()
		self.ngpu = num_gpu
		
		self.conv0 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=1)
		
		self.conv1a = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding = (1, 1))
		self.conv1b = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
		
		self.conv2a = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding = (1, 1))
		self.conv2b = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding = (1, 1))
		
		self.conv3a = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding = (1, 1))
		self.conv3b = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding = (1, 1))
		
		self.pool = nn.MaxPool2d(3, 3)
		
		self.dropout = nn.Dropout2d(p = 0.2)
		
		self.linear1 = nn.Linear(32*10*10, 128)
		self.linear2 = nn.Linear(64*3*3, 128)
		self.linear3 = nn.Linear(128, num_classes)
		
	def forward(self, x):
		x = self.conv0(x)
		x = self.conv1a(x)
		x = self.conv1b(x)
		x = self.pool(x)
		x1 = self.dropout(x)

		x = self.conv2a(x1)
		x = self.conv2b(x)
		x = self.pool(x)
		x2 = self.dropout(x)
	
		x = self.conv3a(x2)
		x = self.conv3b(x)
		x = self.pool(x)
		x3 = self.dropout(x)

		x1 = x1.view(-1, 32*10*10)
		x1 = self.linear1(x1)
		x2 = x2.view(-1, 64*3*3)
		x2 = self.linear2(x2)

		x3 = x3.view(-1, 128*1*1)
		x3 = x1 + x2 + x3
		x3 = self.linear3(x3)
		
		return x3

class TrafficSignClassifier2(nn.Module):
	def __init__(self, num_gpu):
		super(TrafficSignClassifier2, self).__init__()
		self.ngpu = num_gpu
		
		self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=1)
		
		# self.conv2 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
		self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding = (1, 1))
		
		self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding = (2, 2))
		
		self.pool4 = nn.MaxPool2d(3, 3)
		
		self.linear1 = nn.Linear(32*(32*32*3 + 10*10), 128)
		self.linear2 = nn.Linear(128, num_classes)
		
		self.dropout = nn.Dropout(p = 0.2)
		
	def forward(self, x):
		x1 = self.conv1(x)
		x1 = x1.view(32*32*32, -1)
		
		x2 = self.conv1(x)
		x2 = self.conv2(x2)
		x2 = x2.view(32*32*32, -1)
		
		x3 = self.conv1(x)
		x3 = self.conv3(x3)
		x3 = x3.view(32*32*32, -1)
		
		x4 = self.conv1(x)
		x4 = self.conv2(x4)
		x4 = self.pool4(x4)
		x4 = x4.view(32*10*10, -1)
		# print(x1.size(), x2.size(), x3.size(), x4.size())
		
		x = torch.cat((x1, x2, x3, x4), 0)
		x = x.view(-1, 32*(32*32*3 + 10*10))
		
		x = self.linear1(x)
		x = self.dropout(x)
		x = self.linear2(x)
		x = self.dropout(x)
		return x
		
		
print ('...')

...


In [45]:
### Choose different CNN architectures here! ###

# classifier = TrafficSignClassifier(num_gpu).to(device)
# classifier = TrafficSignClassifier1(num_gpu).to(device)
classifier = TrafficSignClassifier2(num_gpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (num_gpu > 0):
    classifier = nn.DataParallel(classifier, list(range(num_gpu)))
    
#To load the trained weights
#Uncomment the following line to load the weights. Then you can run testing directly or continue the training
#classfier.load_state_dict(torch.load('./MP2weights.pth', map_location='cpu')) 
#classfier.eval()

#Print the model
print(classifier)

DataParallel(
  (module): TrafficSignClassifier2(
    (conv1): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (pool4): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (linear1): Linear(in_features=101504, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=43, bias=True)
    (dropout): Dropout(p=0.2)
  )
)


### Set up training environment
You can choose different loss functions and optimizers. Here I just use the same ones as in PyTorch official tutorial.

In [46]:
#training parameters
##TODO
criterion = nn.CrossEntropyLoss()#loss function
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)
####

Now let's start the training. Just like in tutorial, you can print out your loss and the running time at the end of each epoch to monitor the training process. 

In [47]:
def calculate_val_accuracy(valloader):
	correct = 0.
	total = 0.
	predictions = []

	class_correct = list(0. for i in range(num_classes))
	class_total = list(0. for i in range(num_classes))

	for data in valloader:
		images, labels = data
		if num_gpu > 0:
			images = images.cuda()
			labels = labels.cuda()
		outputs = classifier(Variable(images))
		_, predicted = torch.max(outputs.data, 1)
		predictions.extend(list(predicted.cpu().numpy()))
		total += labels.size(0)
		correct += (predicted == labels).sum()

		c = (predicted == labels).squeeze()
		for i in range(len(labels)):
			label = labels[i]
			class_correct[label] += c[i]
			class_total[label] += 1

	class_accuracy = 100.0 * np.divide(class_correct, class_total)
	return 100.0*correct/total, class_accuracy


In [48]:

# Training Loop, no need to run if you already loaded the weights
for epoch in range(40):  # loop over the dataset multiple times  
	running_loss = 0.0
	for i, data in enumerate(trainClassifierLoader, 0):
		# get the inputs
		inputs, labels = data
		if num_gpu > 0:
			inputs = inputs.cuda()
			labels = labels.cuda()
	
		# zero out the parameter gradients
		#Every time a variable is back propogated through, the gradient will be accumulated instead of being replaced. 
		optimizer.zero_grad()
	
		# forward + backward + optimize
		outputs = classifier(inputs)
		loss = criterion(outputs, labels)
	
		#loss.backward() computes dloss/dx for every parameter x
		loss.backward()	
	
		#optimizer.step updates the value of x using the gradient x.grad.
		optimizer.step() 
	
		# print statistics
		running_loss += loss.item()
		
		# if i % 100 == 99:    # print every 2000 mini-batches
		# 	print('[%d, %5d] loss: %.3f' %
		# 		(epoch + 1, i + 1, running_loss / 2000))
		# 	running_loss = 0.0
	
	acc, class_acc = calculate_val_accuracy(trainClassifierLoader)
	print('epoch', epoch, ': loss = ', running_loss, ' accuracy = %d %%' % acc)

print('Finished Training')


epoch 0 : loss =  1119.6301174163818  accuracy = 5 %


epoch 1 : loss =  1102.5690093040466  accuracy = 5 %


epoch 2 : loss =  1099.5631873607635  accuracy = 5 %


epoch 3 : loss =  1096.5573511123657  accuracy = 5 %


epoch 4 : loss =  1096.3902781009674  accuracy = 5 %


...

  File "/home/peixin/miniconda3/envs/py3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


  File "/home/peixin/miniconda3/envs/py3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()


  File "/home/peixin/miniconda3/envs/py3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


  File "/home/peixin/miniconda3/envs/py3.6/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)


  File "/home/peixin/miniconda3/envs/py3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


### Test the Model
Now our model training is finished. If you are satisfied by the result from part of the test set, let's try it on all testing images. Print out the accuracy on all test images. Note you need to achieve 97% accuracy to get full grade.

In [10]:
# classifier.eval()
	
# correct = 0
# total = 0
# predictions = []
# 
# for data in testClassifierLoader:
# 	inputs, labels = data
# 
# 	if num_gpu > 0:
# 		inputs = inputs.cuda()
# 		labels = labels.cuda()
# 
# 	outputs = classifier(Variable(inputs))
# 	_, predicted = torch.max(outputs.data, 1)
# 
# 	total += labels.size(0)
# 	correct += (predicted == labels).sum().item()
# 
# print("accuracy = ", correct / total * 100.0, "%")
# correct = 0.0
# total = 0.0
# with torch.no_grad():
# 	for data in testClassifierLoader:
# 		images, labels = data
# 		if num_gpu > 0:
# 			images = images.cuda()
# 			labels = labels.cuda()
# 		outputs = classifier(images)
# 		_, predicted = torch.max(outputs.data, 1)
# 		total += labels.size(0)
# 		# print('predicted:', predicted)
# 		# print('labels: ', labels)
# 		correct += (predicted == labels).sum().item()
# print(correct, total)
# print('Accuracy of the network on the test images: %d %%' % (
#     100.0 * correct / total))
test_acc, test_class_acc = calculate_val_accuracy(testClassifierLoader)
print('accuracy = %d %%' % test_acc)

	

accuracy = 99 %


### Analyze the Results for Each Individual Class
To help futher improve the accuracy, we want to know for which classes our NN works well and for which classes it fails. Print out the accuracy of your classifier on each class on the whole testing dataset. Try to explain why some classess have low accuracy in your report. You don't have to speficy the name of each class. Just use something like "class 0" is good enough. 

In [11]:
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))
classes = list(range(num_classes))
with torch.no_grad():
	for data in testClassifierLoader:
		images, labels = data
		images = images.cuda()
		labels = labels.cuda()
		outputs = classifier(images)
		_, predicted = torch.max(outputs, 1)
		c = (predicted == labels).squeeze()
		for i in range(labels.size(0)):
			label = labels[i]
			class_correct[label] += c[i].item()
			class_total[label] += 1


for i in range(num_classes):
	print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     0 : 98 %
Accuracy of     1 : 97 %
Accuracy of     2 : 99 %
Accuracy of     3 : 96 %
Accuracy of     4 : 97 %
Accuracy of     5 : 96 %
Accuracy of     6 : 100 %
Accuracy of     7 : 98 %
Accuracy of     8 : 96 %
Accuracy of     9 : 99 %
Accuracy of    10 : 99 %
Accuracy of    11 : 99 %
Accuracy of    12 : 99 %
Accuracy of    13 : 99 %
Accuracy of    14 : 99 %
Accuracy of    15 : 100 %
Accuracy of    16 : 100 %
Accuracy of    17 : 100 %
Accuracy of    18 : 99 %
Accuracy of    19 : 100 %
Accuracy of    20 : 98 %
Accuracy of    21 : 98 %
Accuracy of    22 : 100 %
Accuracy of    23 : 98 %
Accuracy of    24 : 98 %
Accuracy of    25 : 100 %
Accuracy of    26 : 96 %
Accuracy of    27 : 100 %
Accuracy of    28 : 98 %
Accuracy of    29 : 96 %
Accuracy of    30 : 99 %
Accuracy of    31 : 98 %
Accuracy of    32 : 100 %
Accuracy of    33 : 100 %
Accuracy of    34 : 99 %
Accuracy of    35 : 98 %
Accuracy of    36 : 99 %
Accuracy of    37 : 100 %
Accuracy of    38 : 99 %
Accuracy of   

### Save Trained Weights
Notice that the trained weights are just variables right now and will be lost when you close the Jupyter file. Obviously, you don't want to train the model again and again. PyTorch can help you save your work. Read the "Saving & Loading Model for Inference" part from this tutorial: https://pytorch.org/tutorials/beginner/saving_loading_models.html.<br>
Please save the weights from your best model into "MP2weights.pth" and include it in your submission. TAs will not train the CNN for you. So if we cannot find this file, you will lose a lot of points.  

In [12]:
#save the weights into "./MP2weights.pth"
torch.save(classifier.state_dict(), "./cnn_residual.pth")

### Tips for Advanced Techniques
Congratulations! Now we have built a wonderful trffic sign classifier. If you are unsatified about the accuraccy. Here are a few tricks that can help you improve the results:<br>
<li>Balance the dataset. Currently, the size of image dataset from different types of traffic signs vary from 100 to 2000. Hence, our CNN will be trained to be very good at classify those types with a lot of image samples, but behave poorly on the rest of types. One possible solution is to generate more image data from the "unfavorable" types to make each type to have similar number of image samples. </li>
<li>The selection of loss functions and optimizaters  and the learning rate could make a difference. Here is a good reference for different types of loss functions: https://isaacchanghau.github.io/post/loss_functions/</li>
<li>Training the model for too long will make our CNN overfit the training set. If that is the case, we could change the number of epoches to avoid it. </li>
<li>There are piles of articles online that teaches you how to improve your CNN. Do some Google search yourself and try implement some interesting ones.</li>

### Break It If You Can!
Now if you already have trained a neural network with accuracy higher than 90%, let's try to break our system by manipulating the test set. <br>
Find the smallest amount of salt and pepper (recall from lecture 2) noise that has to be added to any image in the data set that was classified correctly, for the image with noise to be miss-classified. Write a function to find this smallest noise for mis-classification, for each image class. Then explore how the smallest noise changes across the different classes. Perform the same experiment with gaussian noise instead of salt and pepper noise. What conclusions can you draw about robustness of your classifier from these experiments? <br>
Note: In this part you only need to pick one single image and test your code on it. Please put the original image and the image after adding noises into your report. 

In [ ]:
##TODO

####